In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Or use load_in_8bit=True
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained(
    "openchat/openchat-3.5-0106",
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained("openchat/openchat-3.5-0106")


c:\Users\aakas\OneDrive\Desktop\Coding\healthcare-chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:38<00:00, 12.81s/it]
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [2]:
# apply lora config
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model,lora_config)

In [3]:
# loading the dataset

dataset = load_dataset("nbertagnolli/counsel-chat")
dataset

Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['questionID', 'questionTitle', 'questionText', 'questionLink', 'topic', 'therapistInfo', 'therapistURL', 'answerText', 'upvotes', 'views'],
        num_rows: 2775
    })
})

In [4]:
dataset["train"][0]

{'questionID': 0,
 'questionTitle': 'Do I have too many issues for counseling?',
 'questionText': 'I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling?',
 'questionLink': 'https://counselchat.com/questions/do-i-have-too-many-issues-for-counseling',
 'topic': 'depression',
 'therapistInfo': 'Jennifer MolinariHypnotherapist & Licensed Counselor',
 'therapistURL': 'https://counselchat.com/therapists/jennifer-molinari',
 'answerText': 'It is very common for\xa0people to have multiple issues that they want to (and need to) address in counseling.\xa0 I have had clients ask that same question and through more exploration, there is often an underlying fear that they\xa0 "can\

In [5]:
'''
The dataset isnt in the correct format for training.
we are going to need to preprocess the data.
'''

# Set pad_token to eos_token (a common and safe choice for CausalLMs)
tokenizer.pad_token = tokenizer.eos_token

# Optional: Let the model know about the new padding token if it wasn't already added
model.resize_token_embeddings(len(tokenizer))

def preprocess_function(example):
    question_title = example.get("questionTitle", "") or ""
    question_text = example.get("questionText", "") or ""
    answer_text = example.get("answerText", "") or ""

    prompt = f"### Question: {question_title.strip()}\n{question_text.strip()}\n\n### Answer:\n"
    response = answer_text.strip() + tokenizer.eos_token  # optional: add EOS token

    # Tokenize separately
    prompt_tokens = tokenizer(prompt, truncation=True, max_length=256, padding="max_length")
    response_tokens = tokenizer(response, truncation=True, max_length=256, padding="max_length")

    input_ids = prompt_tokens["input_ids"] + response_tokens["input_ids"]
    attention_mask = prompt_tokens["attention_mask"] + response_tokens["attention_mask"]

    # Set labels: mask out the prompt tokens with -100
    labels = [-100] * len(prompt_tokens["input_ids"]) + response_tokens["input_ids"]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

processed_df = dataset['train'].map(preprocess_function)
processed_df

Dataset({
    features: ['questionID', 'questionTitle', 'questionText', 'questionLink', 'topic', 'therapistInfo', 'therapistURL', 'answerText', 'upvotes', 'views', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2775
})

In [6]:
# # tokenize

# # pad token
# tokenizer.pad_token = tokenizer.eos_token

# def tokenize_function(examples):
#     return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

# tokenized_datasets = processed_df.map(tokenize_function, batched=True)
# tokenized_datasets

In [7]:
# tokenized_datasets = tokenized_datasets.remove_columns([col for col in tokenized_datasets.column_names if col != "input_ids" and col != "attention_mask"])
# tokenized_datasets.set_format("torch")
# tokenized_datasets

In [8]:
import transformers
print(transformers.__version__)

4.57.0


In [9]:
import accelerate
print(accelerate.__version__)

1.10.1


In [10]:
import transformers
print(transformers.__file__)
print(transformers.__version__)


c:\Users\aakas\OneDrive\Desktop\Coding\healthcare-chatbot\.venv\Lib\site-packages\transformers\__init__.py
4.57.0


In [11]:
from transformers import TrainingArguments

print(TrainingArguments.__module__)


transformers.training_args


In [12]:
# Check the signature of TrainingArguments.__init__ to see if it accepts evaluation_strategy
from inspect import signature
print(signature(transformers.TrainingArguments.__init__))

(self, output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: float = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[transformers.trainer_utils.SchedulerType, str] = 'linear', lr_scheduler_kwargs: Union[dict[str, Any], str] = <factory>, warmup_ratio: float = 0.0, warmup_steps: int = 

In [13]:
import torch

print("Torch CUDA version:", torch.version.cuda)
print("Is CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device name:", torch.cuda.get_device_name(0))
else:
    print("CUDA not available")

Torch CUDA version: 11.8
Is CUDA available: True
CUDA device name: NVIDIA GeForce RTX 3060


In [14]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # No masked LM; GPT-style is causal
)

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [16]:
from transformers import TrainingArguments, Trainer

model.to(device)

training_args = TrainingArguments(
    output_dir="./openchat-counsel-chat",
    eval_strategy="no",  # <- changed from "epoch"
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    report_to="none",
    fp16=True,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_df,
    tokenizer=tokenizer,
    data_collator=data_collator
)

C:\Users\aakas\AppData\Local\Temp\ipykernel_37044\1117840334.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [17]:
from transformers.trainer_utils import get_last_checkpoint
import os
output_dir = "./openchat-counsel-chat"

# if os.path.isdir(output_dir):
#     last_checkpoint = get_last_checkpoint(output_dir)
    
# if last_checkpoint is not None:
#     print(f'Resuming from saved checkpoint: {last_checkpoint}')
#     trainer.train(resume_from_checkpoint=last_checkpoint)
# else:
#     trainer.train()
trainer.train()
    

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 32000}.


Step,Training Loss
10,2.437700
20,2.262500
30,2.228100
40,2.172100
50,2.163000
60,2.138100
70,2.159600
80,2.066800
90,2.110400
100,2.119500


TrainOutput(global_step=2082, training_loss=1.9193961258465917, metrics={'train_runtime': 43131.9996, 'train_samples_per_second': 0.193, 'train_steps_per_second': 0.048, 'total_flos': 1.81938229346304e+17, 'train_loss': 1.9193961258465917, 'epoch': 3.0})

In [18]:
model.save_pretrained("./openchat-counsel-chat")
tokenizer.save_pretrained("./openchat-counsel-chat")

('./openchat-counsel-chat\\tokenizer_config.json',
 './openchat-counsel-chat\\special_tokens_map.json',
 './openchat-counsel-chat\\chat_template.jinja',
 './openchat-counsel-chat\\tokenizer.json')

In [19]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32002, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

In [20]:
def generate_response(prompt, max_tokens=200):
    inputs = tokenizer(prompt,return_tensors="pt").to(device)
    
    with torch.no_grad():
        output = model.generate(
            **inputs, 
            max_new_tokens=max_tokens,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )
        
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [21]:
prompt = "I'm feeling very anxious lately. What should I do?"
response = generate_response(prompt)

print("\n🧠 Model response:")
print(response)



🧠 Model response:
I'm feeling very anxious lately. What should I do?

Anxiety can be overwhelming, but here are a few things you can try to help:

1. Deep breathing exercises - breathe in for 5 seconds, hold for 5 seconds, then breathe out for 5 seconds. Repeat for a few minutes or until you feel calmer.

2. Grounding exercises - focus on the things around you. What color is the shirt you're wearing? What does the carpet feel like under your feet? What is the temperature of the air around you? Try to focus on the present moment and distract yourself from your anxious thoughts.

3. Talk to someone - sometimes just talking to someone about what's bothering you can help you feel better. If you don't have someone you feel comfortable talking to, you can always call a hotline or talk to a counselor on sites like 7 Cups.

4. Exercise - taking a walk or doing any type of physical activity
